In [2]:
# Importing libraries

import os
import base64
import numpy as np
import re
import random
import json
import spacy
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import operator
import seaborn
import cv2
from nltk.corpus import wordnet as wn
from num2words import num2words
import csv

import torch
import logging

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
def get_data(file):
    
    list_values = []
    
    with open(file) as f:
        reader = csv.reader(f, delimiter='\n')
        for line in reader:
            for sent in line:
                list_values.append(sent)
        f.close()
    
    return list_values
    
frcnn_data = get_data("frcnn_emb.csv")
desc_data = get_data("desc_emb.csv")

RuntimeError: [enforce fail at CPUAllocator.cpp:71] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 94673922336 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
#word embeddings to match synonyms
#run the similarity between object labels VS noun phrases
# https://www.sbert.net/docs/usage/semantic_textual_similarity.html

def get_embeddings(data_np,data_frcnn):
    """Gets embeddings from dataset.
    
    Parameters
    ----------
    data_np : list
        A list of sentences from R2R dataset.
        
    data_frcnn : list
        A list of attributes and objects from FRCNN.
        
    Returns
    -------
    output_emb : list
        List of sentences and their embedding.
    """
    
    data_nouns = []
        
    for sent in data_np:
        sent = sent.split()
        for idx, ele in enumerate(sent):
            if ele.isdigit():
                ele = num2words(ele)
                sent[idx] = ele
           
        data_nouns.append(" ".join(sent))
            
    
    output_emb = []
    input_emb_np = data_nouns
    input_emb_f = data_frcnn
    
    embeddings_np = model.encode(input_emb_np, convert_to_tensor=True)
    embeddings_f = model.encode(input_emb_f, convert_to_tensor=True)
    
    #Compute cosine-similarits
    cosine_scores = util.cos_sim(embeddings_np, embeddings_f)
    
    #Output the pairs with their score
    for i in range(len(embeddings_np)):
        print("{} \t\t {} \t\t Score: {:.4f}".format(embeddings_np[i], embeddings_f[i], cosine_scores[i][i]))
    
    #Find the pairs with the highest cosine similarity scores
    pairs = []
    for i in range(len(cosine_scores)-1):
        for j in range(i+1, len(cosine_scores)):
            pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
    for pair in pairs[0:10]:
        i, j = pair['index']
        print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))
    
    #return output_emb

get_embeddings(desc_data,frcnn_data)